### Configuración Inicial

In [1]:
!pip install deepctr-torch torch pandas numpy scikit-learn recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.5 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.2 MB/s eta 0:00:00
   ━━

### Instalación de Librerías

In [2]:
import pandas as pd
import numpy as np
import gdown
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM

### Importación de los Datos

In [3]:
gdown.download(id='1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v', output='training_ratings.csv', quiet=False)
gdown.download(id='1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ', output='validation_ratings.csv', quiet=False)

df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v
From (redirected): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v&confirm=t&uuid=f6c03074-eb51-4638-9404-90332938ed7b
To: /content/training_ratings.csv
100%|██████████| 249M/249M [00:04<00:00, 51.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ
To: /content/validation_ratings.csv
100%|██████████| 58.3M/58.3M [00:00<00:00, 92.1MB/s]


### Preprocesamiento de Datos

In [ ]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [ ]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=1000000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")


Tamaño original del training set: 12390406
Tamaño del nuevo training set (muestra): 1000000


### Configuración de Experimentos

In [ ]:
# Renombrar columnas
df_train_sample = df_train_sample.rename(columns={'user': 'userID', 'item': 'itemID'})
df_val = df_val.rename(columns={'user': 'userID', 'item': 'itemID'})

# Ratings >= 7 se consideran positivos (1), el resto negativos (0)
df_train_sample['label'] = (df_train_sample['rating'] >= 7).astype(int)
df_val['label'] = (df_val['rating'] >= 7).astype(int)

# Se usan las columnas 'userID' y 'itemID' de ambos dataframes para asegurar que todos los IDs estén en el codificador
sparse_features = ['userID', 'itemID']
all_data = pd.concat([df_train_sample, df_val], sort=False)

for feat in sparse_features:
    lbe = LabelEncoder()
    all_data[feat] = lbe.fit_transform(all_data[feat])

# Separar de nuevo en train y validation
df_train_processed = all_data.iloc[:len(df_train_sample)]
df_val_processed = all_data.iloc[len(df_train_sample):]


print("\nEjemplo de datos procesados:")
print(df_train_processed.head())


Ejemplo de datos procesados:
         itemID  rating  userID  label
940711    15953     7.0   22583      1
5329044   13607     6.5  127357      0
2133479    1674     6.0   51346      0
8804705   15135     7.5  204273      1
3195583     814     6.5   76681      0


### Predicción de ratings y top N



In [ ]:
n_users = all_data['userID'].nunique()
n_items = all_data['itemID'].nunique()

print(f"Número de usuarios únicos: {n_users}")
print(f"Número de ítems únicos: {n_items}")


# Definir las características de entrada para el modelo
embedding_dim = 16 # Dimensión de los vectores latentes (embeddings)

feature_columns = [
    SparseFeat('userID', vocabulary_size=n_users, embedding_dim=embedding_dim),
    SparseFeat('itemID', vocabulary_size=n_items, embedding_dim=embedding_dim)
]

dnn_feature_columns = feature_columns
linear_feature_columns = feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Número de usuarios únicos: 282812
Número de ítems únicos: 21923


In [ ]:
# Dividir los datos de validación para tener un conjunto de testeo
train_model_input = {name: df_train_processed[name].values for name in feature_names}
train_labels = df_train_processed['label'].values

val_model_input = {name: df_val_processed[name].values for name in feature_names}
val_labels = df_val_processed['label'].values

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = DeepFM(
    linear_feature_columns=linear_feature_columns,
    dnn_feature_columns=dnn_feature_columns,
    task='binary',
    l2_reg_embedding=1e-5,
    device=device
)

model.compile(
    "adam",
    "binary_crossentropy",
    metrics=["binary_accuracy", "auc"],
)

In [ ]:
history = model.fit(
    train_model_input,
    train_labels,
    batch_size=2048,
    epochs=10,
    verbose=2,
    validation_data=(val_model_input, val_labels)
)

print("\n¡Entrenamiento completado!")

cpu
Train on 1000000 samples, validate on 2901744 samples, 489 steps per epoch
Epoch 1/10
93s - loss:  0.4633 - auc:  0.8287 - val_auc:  0.7839
Epoch 2/10
112s - loss:  0.4180 - auc:  0.8643 - val_auc:  0.7774
Epoch 3/10
124s - loss:  0.3877 - auc:  0.8836 - val_auc:  0.7705
Epoch 4/10
117s - loss:  0.3590 - auc:  0.8989 - val_auc:  0.7647
Epoch 5/10
122s - loss:  0.3397 - auc:  0.9084 - val_auc:  0.7613
Epoch 6/10
124s - loss:  0.3249 - auc:  0.9153 - val_auc:  0.7589
Epoch 7/10
119s - loss:  0.3132 - auc:  0.9206 - val_auc:  0.7576
Epoch 8/10
123s - loss:  0.3030 - auc:  0.9253 - val_auc:  0.7550
Epoch 9/10
123s - loss:  0.2934 - auc:  0.9299 - val_auc:  0.7523
Epoch 10/10
128s - loss:  0.2832 - auc:  0.9351 - val_auc:  0.7491

¡Entrenamiento completado!


In [ ]:
from sklearn.metrics import ndcg_score
pred_scores = model.predict(val_model_input, batch_size=2048)

df_eval = pd.DataFrame({
    'userID': df_val_processed['userID'].values,
    'itemID': df_val_processed['itemID'].values,
    'label': val_labels,
    'score': pred_scores.flatten()
})



def precision_recall_at_k(group, k):
    """Calcula Precision@K y Recall@K para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)

    hits = topk['label'].sum()
    total_relevant = group['label'].sum()

    precision = hits / k
    recall = hits / total_relevant if total_relevant > 0 else 0

    return precision, recall

def ndcg_at_k(group, k):
    """Calcula nDCG@K para un solo usuario/grupo."""
    if group['label'].sum() == 0:
        return 0.0

    ranked_group = group.sort_values('score', ascending=False).head(k)

    # nDCG no se puede calcular si hay menos de 2 ítems en la lista.
    if len(ranked_group) < 2:
        return 0.0

    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])

    return ndcg_score(true_relevance, predicted_scores)



K_values = [10]
results = []

print("Calculando métricas de ranking...")

grouped = df_eval.groupby('userID')

for k in K_values:
    metrics = grouped.apply(lambda x: precision_recall_at_k(x, k))
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])

    ndcg_scores = grouped.apply(lambda x: ndcg_at_k(x, k))
    avg_ndcg = np.mean(ndcg_scores)

    results.append({
        'K': k,
        'Precision@K': avg_precision,
        'Recall@K': avg_recall,
        'nDCG@K': avg_ndcg
    })

results_df = pd.DataFrame(results)
print("\n--- Resultados de Evaluación (DeepFM Individual) ---")
print(results_df)

Calculando métricas de ranking...


/tmp/ipython-input-1083453937.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped.apply(lambda x: precision_recall_at_k(x, k))



--- Resultados de Evaluación (DeepFM Individual) ---
    K  Precision@K  Recall@K    nDCG@K
0  10     0.425634  0.842071  0.752818


/tmp/ipython-input-1083453937.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped.apply(lambda x: ndcg_at_k(x, k))


In [ ]:
# --- 1. Creación de Grupos Sintéticos ---

user_counts = df_eval['userID'].value_counts()
valid_users = user_counts[user_counts >= 10].index.tolist()

# Creamos 1000 grupos sintéticos de 3 usuarios cada uno
np.random.seed(42)
num_groups = 1000
group_size = 4
groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]

print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")
print("Ejemplo de un grupo:", groups[0])


all_group_recs = []

for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]

    item_scores_per_group = group_predictions.groupby('itemID').agg(
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),
        # Para el 'ground truth' del grupo, consideramos un ítem relevante
        # si A TODOS los miembros les gustó (label=1).
        # El producto de las etiquetas será 1 solo si todas son 1.
        group_label=('label', lambda x: 1 if all(x == 1) else 0)
    ).reset_index()

    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)

print("\nEjemplo de scores agregados para un ítem y un grupo:")
print(df_group_eval.head())

Se crearon 1000 grupos sintéticos de tamaño 4.
Ejemplo de un grupo: [ 76353 245777  16044 236844]

Ejemplo de scores agregados para un ítem y un grupo:
   itemID  avg_score  min_score  max_score  group_label  group_id
0      12   0.386087   0.386087   0.386087            1         0
1      44   0.996265   0.996265   0.996265            1         0
2     130   0.016516   0.016516   0.016516            1         0
3    1881   0.996816   0.996816   0.996816            1         0
4    2137   0.998120   0.996240   1.000000            1         0


In [ ]:
# --- 1. Evaluar cada estrategia de agregación ---
strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []

for strategy_name, score_column in strategies.items():
    print(f"Evaluando estrategia: {strategy_name}...")

    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])

        ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
        avg_ndcg = np.mean(ndcg_scores)

        group_results.append({
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación (Estrategias Grupales) ---")
print(group_results_df)

print("\n--- Comparativa Final: Individual vs. Grupal (nDCG@10) ---")
ndcg_individual = results_df[results_df['K'] == 10]['nDCG@K'].iloc[0]
print(f"DeepFM Individual: {ndcg_individual:.4f}")

for strategy_name in strategies.keys():
    ndcg_group = group_results_df[(group_results_df['Strategy'] == strategy_name) & (group_results_df['K'] == 10)]['nDCG@K'].iloc[0]
    print(f"{strategy_name}: {ndcg_group:.4f}")

Evaluando estrategia: Average...


/tmp/ipython-input-3909311734.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3909311734.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))


Evaluando estrategia: Least Misery...


/tmp/ipython-input-3909311734.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3909311734.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))


Evaluando estrategia: Most Pleasure...


/tmp/ipython-input-3909311734.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))



--- Resultados de Evaluación (Estrategias Grupales) ---
        Strategy   K  Precision@K  Recall@K    nDCG@K
0        Average  10       0.8872  0.142025  0.960907
1   Least Misery  10       0.8879  0.142149  0.960949
2  Most Pleasure  10       0.8745  0.139873  0.955469

--- Comparativa Final: Individual vs. Grupal (nDCG@10) ---
DeepFM Individual: 0.7528
Average: 0.9609
Least Misery: 0.9609
Most Pleasure: 0.9555


/tmp/ipython-input-3909311734.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
